# Dataset Overview

Quick reference notebook to summarize the raw taxi + Citi Bike data. Run this before writing the report to capture fleet sizes, ride counts, average durations/distances, and station/zone coverage.

In [1]:
import json
from pathlib import Path
import sys

import numpy as np
import pandas as pd

In [2]:
# Add src/ to path for shared loaders
root = Path.cwd().resolve()
for candidate in [root, *root.parents]:
    src_dir = candidate / 'src'
    if src_dir.exists():
        sys.path.append(str(src_dir))
        break

from modeling.travel_diagnostics import load_taxi_trips, load_bike_trips


In [3]:
DATA_ROOT = Path('/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw')
TAXI_PATHS = sorted(DATA_ROOT.glob('yellow_tripdata_2024-*.parquet'))
BIKE_ROOT = DATA_ROOT / 'citibike'
BIKE_GLOB = '20240*-citibike-tripdata_*.csv'
print(f'Taxi files: {len(TAXI_PATHS)}')
print(f'Bike glob: {BIKE_GLOB}')


Taxi files: 6
Bike glob: 20240*-citibike-tripdata_*.csv


## Taxi sample statistics

In [4]:
path = Path("/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw/yellow_tripdata_2024-01.parquet")
df = pd.read_parquet(path)
print(df.head())
print(df.columns)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1         1  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2         1  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   
3         1  2024-01-01 00:36:38   2024-01-01 00:44:56              1.0   
4         1  2024-01-01 00:46:51   2024-01-01 00:52:57              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72         1.0                  N           186            79   
1           1.80         1.0                  N           140           236   
2           4.70         1.0                  N           236            79   
3           1.40         1.0                  N            79           211   
4           0.80         1.0                  N           211           148   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [5]:
taxi_frames = []
for path in TAXI_PATHS:
    print(f'Loading {path.name} ...')
    taxi_frames.append(load_taxi_trips(path, max_rows=None))

taxi = pd.concat(taxi_frames, ignore_index=True)
print(f"Taxi trips after filters: {len(taxi):,}")
if 'PULocationID' in taxi.columns:
    print('Unique pickup zones:', taxi['PULocationID'].nunique())
else:
    print('Unique pickup zones: unavailable (column missing)')
print('Unique hours observed:', taxi['hour'].nunique())

taxi_summary = taxi[['travel_min', 'distance_km', 'is_rush', 'is_weekend']].describe(include='all')
display(taxi_summary)

Loading yellow_tripdata_2024-01.parquet ...
Loading yellow_tripdata_2024-02.parquet ...
Loading yellow_tripdata_2024-03.parquet ...
Loading yellow_tripdata_2024-04.parquet ...
Loading yellow_tripdata_2024-05.parquet ...
Loading yellow_tripdata_2024-06.parquet ...
Taxi trips after filters: 17,550,334
Unique pickup zones: 258
Unique hours observed: 24


,travel_min,distance_km,is_rush,is_weekend
count,1.755033e+07,1.755033e+07,17550334,17550334
unique,NaN,NaN,2,2
top,NaN,NaN,False,False
freq,NaN,NaN,12062480,12624192
mean,1.313588e+01,3.238560e+00,NaN,NaN
std,8.106863e+00,2.310941e+00,NaN,NaN
min,1.000000e+00,1.609340e-02,NaN,NaN
25%,7.266667e+00,1.609340e+00,NaN,NaN
50%,1.141667e+01,2.574944e+00,NaN,NaN
75%,1.715000e+01,4.152097e+00,NaN,NaN


## Citi Bike sample statistics

In [8]:
df = pd.read_csv('/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw/citibike/202401-citibike-tripdata_1.csv')
print(df.head())

            ride_id  rideable_type               started_at  \
0  8E865410DBDE0CA9  electric_bike  2024-01-01 13:00:04.563   
1  0403D0B3FC9CA77D  electric_bike  2024-01-08 19:36:43.520   
2  F6DE7BB42FF550BE  electric_bike  2024-01-12 15:00:41.580   
3  84A995BFD98030D4   classic_bike  2024-01-12 16:52:19.025   
4  7BBEAD4F2B535813  electric_bike  2024-01-05 19:50:19.202   

                  ended_at           start_station_name start_station_id  \
0  2024-01-01 13:04:04.652                 3 St & 3 Ave          4028.03   
1  2024-01-08 19:53:16.266  Franklin Ave & St Marks Ave          4107.05   
2  2024-01-12 15:36:29.622           W 67 St & Broadway          7116.04   
3  2024-01-12 17:17:29.773  Central Park West & W 68 St          7079.06   
4  2024-01-05 20:34:42.517           W 67 St & Broadway          7116.04   

            end_station_name end_station_id  start_lat  start_lng    end_lat  \
0      Carroll St & Smith St        4225.14  40.675070 -73.987752  40.680611   
1   

/var/folders/hw/zpghgr8x4ybc9d8tnpvwmn780000gn/T/ipykernel_19486/3939737073.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/atharvramesh/UCSD/Fall2025/ECE225A/NYC_Public_Transit/data/raw/citibike/202401-citibike-tripdata_1.csv')


In [6]:
bike = load_bike_trips(BIKE_ROOT, BIKE_GLOB, max_rows=None)
print(f"Bike trips after filters: {len(bike):,}")
print('Unique start stations:', bike['rideable_type'].nunique())
print('Rideable mix:', bike['rideable_type'].value_counts())

bike_summary = bike[['travel_min', 'distance_km', 'rideable_type', 'is_rush', 'is_weekend']].describe(include='all')
display(bike_summary)


Bike trips after filters: 18,253,964
Unique start stations: 2
Rideable mix: rideable_type
electric_bike    11854157
classic_bike      6399807
Name: count, dtype: int64


,travel_min,distance_km,rideable_type,is_rush,is_weekend
count,1.825396e+07,1.825396e+07,18253964,18253964,18253964
unique,NaN,NaN,2,2,2
top,NaN,NaN,electric_bike,False,False
freq,NaN,NaN,11854157,10745186,13527779
mean,1.209192e+01,2.024503e+00,NaN,NaN,NaN
std,1.080158e+01,1.671537e+00,NaN,NaN,NaN
min,1.000383e+00,5.373035e-04,NaN,NaN,NaN
25%,5.186800e+00,8.744288e-01,NaN,NaN,NaN
50%,8.890842e+00,1.511654e+00,NaN,NaN,NaN
75%,1.533917e+01,2.628416e+00,NaN,NaN,NaN
